<a href="https://colab.research.google.com/github/sammy22/sudoku-CNN/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [313]:
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import tensorflow as tf

clone the repo to load the images and model 


In [315]:
!git clone https://github.com/sammy22/sudoku-CNN.git

Cloning into 'sudoku-CNN'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 40 (delta 11), reused 25 (delta 4), pack-reused 0
Unpacking objects: 100% (40/40), done.


Read and display the sudoku image 

In [348]:
# img = cv2.imread('./sudoku-CNN/images/img_15.png')
img = cv2.imread('./sudoku-CNN/images/img_10.jpeg')
# cv2_imshow(img)

Convert image to grayscale and apply thresholding 

In [349]:
width,height = 450,450 # to have all image of same size
img = cv2.resize(img, (width, height))  
imgBlank = np.zeros((height, width, 3), np.uint8)  # for testing
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) 
imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, 1, 1, 11, 2)  
# cv2_imshow(imgThreshold)

Some images of sudoku have gaps between images. To fix this we use morphological transformation. 

In [ ]:
# kernelSize = (3,3)
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernelSize)
# imgThreshold = cv2.morphologyEx(imgThreshold, cv2.MORPH_GRADIENT, kernel)
# cv2_imshow(imgThreshold)

Finding the biggest contour in an image. as thats the boundary of sudoku. 

In [350]:
imgContours = img.copy() 
contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # FIND ALL CONTOURS
cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 3) # DRAW ALL DETECTED CONTOURS
# cv2_imshow(imgContours)

biggest,max_area = np.array([]),0
for i in contours:
    area = cv2.contourArea(i)    
    if area > 50:
        approx = cv2.approxPolyDP(i, 0.02 * cv2.arcLength(i, True), True)
        if area > max_area and len(approx) == 4:
            biggest,max_area = approx,area
#print(biggest)

split image into 81 images, one image for every cell 

In [351]:
def reorder(myPoints):
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1) # add x and y coordinates
    myPointsNew[0],myPointsNew[3] = myPoints[np.argmin(add)],myPoints[np.argmax(add)] #topleft and bottom right 
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1],myPointsNew[2] =myPoints[np.argmin(diff)],myPoints[np.argmax(diff)] #topright and bottom left
    return myPointsNew

imgBigContour = img.copy() 
if biggest.size != 0:
    biggest = reorder(biggest)
    cv2.drawContours(imgBigContour, biggest, -1, (0, 0, 255), 25) 
    # cv2_imshow(imgBigContour)
    pts1 = np.float32(biggest) 
    pts2 = np.float32([[0, 0],[width, 0], [0, height],[width, height]]) 
    matrix = cv2.getPerspectiveTransform(pts1, pts2) # GER
    imgWarpColored = cv2.warpPerspective(img, matrix, (width, height))
    imgDetectedDigits = imgBlank.copy()
    imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
    # cv2_imshow(imgWarpColored)
    boxes=[]
    for rows in np.vsplit(imgWarpColored,9):
        for box in np.hsplit(rows,9):
            boxes.append(box)
    #cv2_imshow(boxes[55])

Load model trained on different epochs. Whichever works best for the image 

In [352]:
model = tf.keras.models.load_model('./sudoku-CNN/mnist_10.h5')
# model = tf.keras.models.load_model('./sudoku-CNN/mnist_15.h5') 
# model = tf.keras.models.load_model('./sudoku-CNN/mnist_30.h5')

Predict each cell and generate the sudoku as list 

In [353]:
predictions = []
for image in boxes:
    im = np.asarray(image)
    im = im[5:im.shape[0] - 5, 5:im.shape[1]-5]
    im = cv2.resize(im, (28, 28))
    im = cv2.bitwise_not(im)     
    if np.sum(img >200)==0:
      predictions.append(0)
      continue
    im = im.reshape(1, 28, 28, 1)/255
    predictions.append(np.argmax(model.predict(im),axis=1)[0])

sudoku_array=[[predictions[9*i+j] for j in range(9)] for i in range(9)]

In [355]:
# print(np.array(sudoku_array))
# cv2_imshow(img)